In [2]:
# 加载包
import requests
from bs4 import BeautifulSoup
import csv
import time
import pandas as pd

In [4]:
# 参数设置
base_url = 'http://www.ochirly.com.cn'
url_mid = 'list-'
url_end = '-40-listedDate%20desc-0-0-0-0-0.shtml'
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 UBrowser/6.1.2107.204 Safari/537.36'}

In [39]:
# 读取商品分类信息:ochirly_category.csv
ochirly_category = pd.read_table('./ochirly_category.csv', 
                                 sep = ',', 
                                 encoding = 'gbk')
ochirly_category

,cat1,cat2,url,page_num
0,场景,休闲,/Scene/Scene_Casual/,1
1,场景,工作,/Scene/Scene_Work/,2
2,场景,逛街约会,/Scene/Scene_Date/,1
3,场景,度假,/Scene/Scene_Holiday/,1
4,场景,派对,/Scene/Scene_Party/,1
5,场景,SNOOPY限量版,/Scene/Scene_Snoopy/,1
6,新品,9月19日到店,/New_In/New_in_9_19/,2
7,新品,9月12日到店,/New_In/New_in_9_12/,2
8,新品,9月5日到店,/New_In/New_in_9_5/,1
9,服装,毛织,/Clothing/Sweaters/,15


In [64]:
# 爬取商品的场景分类
## 从ochirly_category表中筛选场景购的信息
ochirly_changjing = ochirly_category.loc[ochirly_category.cat1 == '场景',]
## 重置row index为从0开始
ochirly_changjing = ochirly_changjing.reset_index(drop = True)
## 打开一个 csv 文件，把goods_id 和场景名称写入
csv_file = open('./ochirly_changjing_goods.csv', 
                'w', 
                newline = '', 
                encoding = 'utf-8-sig')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['goods_id', 'changjing'])
## 获取 goods_id 和场景名称
for row_num in range(ochirly_changjing.shape[0]):
    changjing = ochirly_changjing.loc[row_num, 'cat2']
    for page_num in range(ochirly_changjing.loc[row_num, 'page_num']):
        url1 = base_url + ochirly_changjing.loc[row_num, 'url'] + url_mid + str(page_num+1) + url_end
        page1 = requests.get(url1, headers = header)
        source1 = BeautifulSoup(page1.content, 'html.parser', from_encoding = 'gb18030')
        goods_infos = source1.find('div', class_ = 'product_list').find_all('li')
        for goods_info in goods_infos:
            goods_href = goods_info.find('a').get('href')
            url2 = base_url + goods_href
            page2 = requests.get(url2, headers = header)
            source2 = BeautifulSoup(page2.content, 'html.parser', from_encoding = 'gb18030')
            goods_id = source2.find('p', class_ = 'number').string.replace('编号 ', '')
            ## 把结果写入到 csv 文件中
            csv_writer.writerow([goods_id, changjing])
        # 休息一下
        time.sleep(3)
# 关闭 csv_file
csv_file.close()

In [69]:
# 爬取商品的上市日期
## 从ochirly_category表中筛选新品
ochirly_newin = ochirly_category.loc[ochirly_category.cat1 == '新品',]
## 重置row index为从0开始
ochirly_newin = ochirly_newin.reset_index(drop = True)
## 打开一个 csv 文件，把goods_id 和上市日期写入
csv_file = open('./ochirly_newin_goods.csv', 
                'w', 
                newline = '', 
                encoding = 'utf-8-sig')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['goods_id', 'newin_date'])
## 获取 goods_id 和上市日期
for row_num in range(ochirly_newin.shape[0]):
    newin_date = ochirly_newin.loc[row_num, 'cat2'].replace('到店', '').replace('月', '-').replace('日','')
    for page_num in range(ochirly_newin.loc[row_num, 'page_num']):
        url1 = base_url + ochirly_newin.loc[row_num, 'url'] + url_mid + str(page_num+1) + url_end
        page1 = requests.get(url1, headers = header)
        source1 = BeautifulSoup(page1.content, 'html.parser', from_encoding = 'gb18030')
        goods_infos = source1.find('div', class_ = 'product_list').find_all('li')
        for goods_info in goods_infos:
            goods_href = goods_info.find('a').get('href')
            url2 = base_url + goods_href
            page2 = requests.get(url2, headers = header)
            source2 = BeautifulSoup(page2.content, 'html.parser', from_encoding = 'gb18030')
            goods_id = source2.find('p', class_ = 'number').string.replace('编号 ', '')
            ## 把结果写入到 csv 文件中
            csv_writer.writerow([goods_id, newin_date])
        # 休息一下
        time.sleep(3)
# 关闭 csv_file
csv_file.close()

AttributeError: 'NoneType' object has no attribute 'find_all'

In [ ]:
# ==================================================================
# 爬取所有服装商品 ===================================================
# ==================================================================
## 从ochirly_category表中筛选服装的信息
ochirly_yifu = ochirly_category.loc[ochirly_category.cat1 == '服装',]
## 重置row index为从0开始
ochirly_yifu = ochirly_yifu.reset_index(drop = True)
## 打开一个 csv 文件，写入商品信息
csv_file = open('./ochirly_goods_info.csv', 
                'w', 
                newline = '', 
                encoding = 'utf-8-sig')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['category', 'goods_id', 'goods_name', 'color', 'sale_price', 
                     'origin_price', 'design', 'material', 'stereotype', 'length'])
## 获取商品信息
for row_num in range(ochirly_yifu.shape[0]):
    category = ochirly_yifu.loc[row_num, 'cat2']
    for page_num in range(ochirly_yifu.loc[row_num, 'page_num']):
        url1 = base_url + ochirly_yifu.loc[row_num, 'url'] + url_mid + str(page_num+1) + url_end
        page1 = requests.get(url1, headers = header)
        source1 = BeautifulSoup(page1.content, 'html.parser', from_encoding = 'gb18030')
        goods_infos = source1.find('div', class_ = 'product_list').find_all('li')
        for goods_info in goods_infos:
            goods_href = goods_info.find('a').get('href')
            url2 = base_url + goods_href
            page2 = requests.get(url2, headers = header)
            source2 = BeautifulSoup(page2.content, 'html.parser', from_encoding = 'gb18030')
            goods_id = source2.find('p', class_ = 'number').string.replace('编号 ', '')
            try:
                goods_name = source2.find('p', class_ = 'title').string.replace('\n', '').replace('	', '')
            except Exception as e:
                goods_name = None
            try:
                color = source2.find('div', class_ = 'color clearfix').a.find('div', class_ = 'arrow').p.string
            except Exception as e:
                color = None
            try:
                price = source2.find('p', class_ = 'price').em.children
                sale_price = next(price).replace('￥','')
                origin_price = next(price).string.replace('￥','')
            except Exception as e:
                sale_price = source2.find('p', class_ = 'price').string.replace('\n', '').replace('	', '').replace('￥','')
                origin_price = source2.find('p', class_ = 'price').string.replace('\n', '').replace('	', '').replace('￥','')
            try:
                design = source2.find(attrs = {'id':'detailContent1'}).find_all('li')[0].p.string
            except Exception as e:
                design = None
            try:
                material = source2.find(attrs = {'id':'detailContent1'}).find_all('li')[1].p.string.replace('面料:','')
            except Exception as e:
                material = None
            try:
                stereotype = source2.find(attrs = {'id':'detailContent2'}).find_all('li')[0].p.string
            except Exception as e:
                stereotype = None
            try:
                length = source2.find(attrs = {'id':'detailContent2'}).find_all('li')[1].p.string
            except Exception as e:
                length = None
            ## 把结果写入到 csv 文件中
            csv_writer.writerow([category, goods_id, goods_name, color, sale_price, 
                                 origin_price, design, material, stereotype, length])
            
            ## 由于图片比较耗资源，暂时不爬取图片
            '''
            ## 爬取主图片
            main_src = source2.find('ul', class_ = 'lineone').find('div', class_ = 'big_img').find('img').get('data-src')
            img = requests.get(main_src, timeout = 3)
            img_name = 'ochirly_goods_image/' + goods_id + '.jpg'
            fp = open(img_name, 'wb')
            fp.write(img.content)
            fp.close()
            time.sleep(2)
            ## 爬取展示图片
            show_src1 = source2.find('ul', class_ = 'linetwo clearfix').find_all('li')[0].find('img').get('data-src')
            img = requests.get(show_src1, timeout = 3)
            img_name = 'ochirly_goods_image/' + goods_id + '-show1' + '.jpg'
            fp = open(img_name, 'wb')
            fp.write(img.content)
            fp.close()
            time.sleep(2)
            show_src2 = source2.find('ul', class_ = 'linetwo clearfix').find_all('li')[1].find('img').get('data-src')
            img = requests.get(show_src2, timeout = 3)
            img_name = 'ochirly_goods_image/' + goods_id + '-show2' + '.jpg'
            fp = open(img_name, 'wb')
            fp.write(img.content)
            fp.close()
            time.sleep(2)
            show_src3 = source2.find('ul', class_ = 'linetwo clearfix').find_all('li')[2].find('img').get('data-src')
            img = requests.get(show_src3, timeout = 3)
            img_name = 'ochirly_goods_image/' + goods_id + '-show3' + '.jpg'
            fp = open(img_name, 'wb')
            fp.write(img.content)
            fp.close()
            time.sleep(2)
            show_src4 = source2.find('ul', class_ = 'linetwo clearfix').find_all('li')[3].find('img').get('data-src')
            img = requests.get(show_src4, timeout = 3)
            img_name = 'ochirly_goods_image/' + goods_id + '-show4' + '.jpg'
            fp = open(img_name, 'wb')
            fp.write(img.content)
            fp.close()
            time.sleep(2)
            ## 爬取细节图片
            detail_src1 = source2.find('ul', class_ = 'linethree clearfix').find_all('li')[0].find('img').get('data-src')
            img = requests.get(detail_src1, timeout = 3)
            img_name = 'ochirly_goods_image/' + goods_id + '-detail1' + '.jpg'
            fp = open(img_name, 'wb')
            fp.write(img.content)
            fp.close()
            time.sleep(2)
            detail_src2 = source2.find('ul', class_ = 'linethree clearfix').find_all('li')[1].find('img').get('data-src')
            img = requests.get(detail_src2, timeout = 3)
            img_name = 'ochirly_goods_image/' + goods_id + '-detail2' + '.jpg'
            fp = open(img_name, 'wb')
            fp.write(img.content)
            fp.close()
            time.sleep(2)
            detail_src3 = source2.find('ul', class_ = 'linethree clearfix').find_all('li')[2].find('img').get('data-src')
            img = requests.get(detail_src3, timeout = 3)
            img_name = 'ochirly_goods_image/' + goods_id + '-detail3' + '.jpg'
            fp = open(img_name, 'wb')
            fp.write(img.content)
            fp.close()
            time.sleep(2)
            '''
        # 显示进度
        print(category, '第', page_num+1, '页')
        print('------------')
        # 每爬取一页，休息一下
        time.sleep(30)
# 没爬取一个品类，休息一下
time.sleep(300)
# 关闭 csv_file
csv_file.close()

毛织 第 1 页
------------
毛织 第 2 页
------------
毛织 第 3 页
------------
毛织 第 4 页
------------
毛织 第 5 页
------------
毛织 第 6 页
------------
毛织 第 7 页
------------
毛织 第 8 页
------------
毛织 第 9 页
------------
毛织 第 10 页
------------
毛织 第 11 页
------------
毛织 第 12 页
------------


In [38]:
# ==================================================================
# 连续爬取多个品类中途容易中断，因此单独爬取一个品类 =======================
# ==================================================================
## 选择需要爬取的品类
ochirly_yifu = ochirly_category.loc[ochirly_category.cat2 == '毛织',]
## 重置row index为从0开始
ochirly_yifu = ochirly_yifu.reset_index(drop = True)
## 打开一个 csv 文件，写入商品信息
csv_file = open('./ochirly_goods_info.csv', 
                'w', 
                newline = '', 
                encoding = 'utf-8-sig')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['category', 'goods_id', 'goods_name', 'color', ' sale_price', 
                     'origin_price', 'design', 'material', 'stereotype', 'length'])
## 获取商品信息
category = ochirly_yifu.loc[0, 'cat2']
for page_num in range(ochirly_yifu.loc[0, 'page_num']):
    url1 = base_url + ochirly_yifu.loc[0, 'url'] + url_mid + str(page_num+1) + url_end
    page1 = requests.get(url1, headers = header)
    source1 = BeautifulSoup(page1.content, 'html.parser', from_encoding = 'gb18030')
    goods_infos = source1.find('div', class_ = 'product_list').find_all('li')
    for goods_info in goods_infos:
        goods_href = goods_info.find('a').get('href')
        url2 = base_url + goods_href
        page2 = requests.get(url2, headers = header)
        source2 = BeautifulSoup(page2.content, 'html.parser', from_encoding = 'gb18030')
        goods_id = source2.find('p', class_ = 'number').string.replace('编号 ', '')
        try:
            goods_name = source2.find('p', class_ = 'title').string.replace('\n', '').replace('	', '')
        except Exception as e:
            goods_name = None
        try:
            color = source2.find('div', class_ = 'color clearfix').a.find('div', class_ = 'arrow').p.string
        except Exception as e:
            color = None
        try:
            price = source2.find('p', class_ = 'price').em.children
            sale_price = next(price).replace('￥','')
            origin_price = next(price).string.replace('￥','')
        except Exception as e:
            sale_price = source2.find('p', class_ = 'price').string.replace('\n', '').replace('	', '').replace('￥','')
            origin_price = source2.find('p', class_ = 'price').string.replace('\n', '').replace('	', '').replace('￥','')
        try:
            design = source2.find(attrs = {'id':'detailContent1'}).find_all('li')[0].p.string
        except Exception as e:
            design = None
        try:
            material = source2.find(attrs = {'id':'detailContent1'}).find_all('li')[1].p.string.replace('面料:','')
        except Exception as e:
            material = None
        try:
            stereotype = source2.find(attrs = {'id':'detailContent2'}).find_all('li')[0].p.string
        except Exception as e:
            stereotype = None
        try:
            length = source2.find(attrs = {'id':'detailContent2'}).find_all('li')[1].p.string
        except Exception as e:
            length = None
        ## 把结果写入到 csv 文件中
        csv_writer.writerow([category, goods_id, goods_name, color, sale_price, 
                             origin_price, design, material, stereotype, length])

    # 显示进度
    print(category, '第', page_num+1, '页')
    print('------------')
    # 休息一下
    time.sleep(30)
# 关闭 csv_file
csv_file.close()

毛织 第 1 页
------------


ConnectionError: HTTPConnectionPool(host='www.ochirly.com.cn', port=80): Max retries exceeded with url: /p/1GZ3031740950.shtml (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x10b808400>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known',))

In [35]:
url2 = 'http://www.ochirly.com.cn/p/1JY4081920090.shtml'
page2 = requests.get(url2, headers = header)
source2 = BeautifulSoup(page2.content, 'html.parser', from_encoding = 'gb18030')
goods_id = source2.find('p', class_ = 'number').string.replace('编号 ', '')
price = source2.find('p', class_ = 'price').em.children
sale_p = next(price)
origin_p = next(price).string
print(sale_p)
print(origin_p)
print('====')

￥489 
￥969
====


In [36]:
url2 = 'http://www.ochirly.com.cn/p/1JY4081920090.shtml'
page2 = requests.get(url2, headers = header)
source2 = BeautifulSoup(page2.content, 'html.parser', from_encoding = 'gb18030')
goods_id = source2.find('p', class_ = 'number').string.replace('编号 ', '')
try:
    goods_name = source2.find('p', class_ = 'title').string.replace('\n', '').replace('	', '')
except Exception as e:
    goods_name = None
try:
    color = source2.find('div', class_ = 'color clearfix').a.find('div', class_ = 'arrow').p.string
except Exception as e:
    color = None
try:
    price = source2.find('p', class_ = 'price').em.children
    sale_price = next(price).replace('￥','')
    origin_price = next(price).string.replace('￥','')
except Exception as e:
    sale_price = source2.find('p', class_ = 'price').string.replace('\n', '').replace('	', '').replace('￥','')
    origin_price = source2.find('p', class_ = 'price').string.replace('\n', '').replace('	', '').replace('￥','')
try:
    design = source2.find(attrs = {'id':'detailContent1'}).find_all('li')[0].p.string
except Exception as e:
    design = None
try:
    material = source2.find(attrs = {'id':'detailContent1'}).find_all('li')[1].p.string.replace('面料:','')
except Exception as e:
    material = None
try:
    stereotype = source2.find(attrs = {'id':'detailContent2'}).find_all('li')[0].p.string
except Exception as e:
    stereotype = None
try:
    length = source2.find(attrs = {'id':'detailContent2'}).find_all('li')[1].p.string
except Exception as e:
    length = None

print(sale_price)
print(origin_price)

489 
969


In [101]:
url1 = 'http://www.ochirly.com.cn/p/1GY4330780532.shtml'
page1 = requests.get(url1, headers = header)
source1 = BeautifulSoup(page1.content, 'html.parser', from_encoding = 'gb18030')
img_src = source1.find('ul', class_ = 'lineone')
print(img_src)

<ul class="lineone">
<li>
<div class="zoom_wrap" id="zoomWrap">
<div class="middle_img" id="middleImg">
<img alt="翻领长款羽绒服 " data-src="http://img1.ochirly.com.cn/wcsstore/TrendyCatalogAssetStore/images/trendy/ochirly/2018/d/1GY4330780532/1GY4330780532_m_8.jpg" src="//img2.ochirly.com.cn/rs/common/images/blank.gif"/>
</div>
<div class="big_img" id="bigImg">
<img alt="翻领长款羽绒服 " data-src="http://img1.ochirly.com.cn/wcsstore/TrendyCatalogAssetStore/images/trendy/ochirly/2018/d/1GY4330780532/1GY4330780532_b_8.jpg" src="//img2.ochirly.com.cn/rs/common/images/blank.gif"/>
</div>
</div>
</li>
</ul>


In [51]:
print(ochirly_newin.loc[7,'cat2'])

9月5日到店


In [48]:
ochirly_newin = ochirly_category.loc[ochirly_category.cat1 == '新品',]
print(ochirly_newin.shape)
## 获取 goods_id 和场景名称
for row_num in range(ochirly_newin.shape[0]):
    #newin_date = ochirly_newin.loc[row_num, 'cat2']
    print(row_num)
    #print(newin_date)

(5, 4)
0
1
2
3
4


In [35]:
url2 = 'http://www.ochirly.com.cn/p/1GH2085730510.shtml'
page2 = requests.get(url2, headers = header)
source2 = BeautifulSoup(page2.content, 'html.parser', from_encoding = 'gb18030')
goods_id = source2.find('p', class_ = 'number').string.replace('编号 ', '')
print(goods_href)
print(goods_id)
print(goods_price)

/p/1GY1084160090.shtml
1GH2085730510
						


In [44]:
ochirly_changjing = ochirly_category.loc[ochirly_category.cat1 == '场景',]
## 获取 goods_id 和场景名称
for row_num in range(ochirly_changjing.shape[0]):
    changjing = ochirly_changjing.loc[row_num, 'cat2']
    print(row_num)
    print(changjing)

0
休闲
1
工作
2
逛街约会
3
度假
4
派对
5
SNOOPY限量版
